In [5]:
from numpy.core.fromnumeric import size
import cv2
import numpy as np
import imutils
from imutils.video import FPS
from keras.models import load_model
import tensorflow as tf
from keras.preprocessing import image
import pandas as pd 

#set argument

INPUT = 'demo.mp4'
OUTPUT = 'output.avi'
LABEL = 'C:/Users/CS-Guest-2/Desktop/Nuig/MSc_AI/MSc-AI/Research_Topics_AI/Assignment1/yolov3.txt'
FILE = 'cars.txt'
CONFIG = 'yolo3.cfg'
WEIGHT  = 'yolo3.weights'


#load model

model = load_model('C:/Users/CS-Guest-2/Desktop/Nuig/MSc_AI/MSc-AI/Research_Topics_AI/object-detection-opencv/saved_model')


#capture frame from input

vs = cv2.VideoCapture(INPUT)
writer = None
fps = FPS().start()
countSedan = 0
countsuv = 0
Width = None
Height = None
cars = {'Total' : [] , 'Sedan' : [] , 'SUV' : []}

#write the frames to video
writer = cv2.VideoWriter(OUTPUT,cv2.VideoWriter_fourcc(*'MJPG'),30,(800,600),1)

def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return layer_names



def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h,countSedan,countsuv):
   
    with open(FILE, 'r') as f:
        class1 = [line.strip() for line in f.readlines()]
    label = str(class1[0])
    label1 = str(class1[1])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    
    new_img = np.expand_dims(img,axis =0)
    pred = model.predict(new_img)
    ans =pred.argmax(axis = -1)
    if ans[0] == 0:
        cv2.putText(img, label, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        countSedan +=1
    else:
        cv2.putText(img, label1, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        countsuv +=1
    return countSedan, countsuv
        

classes = None

with open(LABEL, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
 # Initialize netwrok and load weights
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
  




while True:
    try:
        (grabbed,frame) = vs.read()
    except:
        break

    if not grabbed:
        break
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)

    if Width is None or Height is None:
        Height,Width = frame.shape[:2]
    outs = net.forward(get_output_layers(net))


    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.8
    nms_threshold = 0.4


    for out in outs:
        for detection in out:

            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                box = detection[0:4] * np.array([Width,Height,Width,Height])
                (center_x,center_y,w,h) = box.astype("int")

                x = center_x - w / 2
                y = center_y - h / 2

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, int(w), int(h)])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    # for each frame predict the object detected and draw boundary 
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]        
        countSedan,countsuv =  draw_prediction(frame, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h),countSedan,countsuv)
    cv2.putText(frame, f'count = {len(indices)} cars' , (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.putText(frame, f'Sedan Cars  = {countSedan}' , (50,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.putText(frame, f'Suv Cars  = {countsuv}' , (50,90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cars["Total"].append(len(indices))
    cars["Sedan"].append(countSedan)
    cars["SUV"].append(countsuv)
    df = pd.DataFrame(cars, columns = ['Total', 'Sedan','Suv'])   
    cv2.imshow("output",cv2.resize(frame,(800,600)))
    fps.update()
    countSedan = 0
    countsuv = 0
    writer.write(cv2.resize(frame,(800,600)))
    if cv2.waitKey(1) & 0xFF == ord('s'): 
        break
fps.stop()
df.index +=1
df.to_csv('groundtruth.csv')
cv2.destroyAllWindows()
writer.release()
vs.release()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20201117::readNetFromDarknet'


In [26]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

df1 = pd.read_csv('groundtruth.csv')
df2 = pd.read_excel('Groundtruth.xlsx')

#f1_score(df2['Total'], df1['Total'], average  = 'weighted')
#accuracy_score(df2['Total'], df1['Total'])
confusion_matrix(df2['Total'], df1['Total'])

array([[  0,   0,   0,   0,   0,   0],
       [  1,  31,  41,   1,   5,   1],
       [ 31,  50,  54,   3,   4,   0],
       [  1,  66,  83,  65,  25,   1],
       [  3,  63, 138, 128,  43,   8],
       [  0,   0,   8,  15,  26,   5]], dtype=int64)